In [206]:
from IPython.core.display import display, HTML
from string import Template

## Output HTML from Python

Instead of raw text output, we can output rich HTML...

In [207]:
display(HTML('<h1>Hello, world!</h1>'))

We can also output the other things we might use in developing a webpage, like CSS:

In [208]:
display(HTML('''
<style>
    h1:hover {
      background-color: yellow;
    }
</style>
<h1>Hello, world!</h1>
'''))

Note that CSS is global, so even though we may have run this after outputting our first `h1` tag, it affects all `h1` tags in the document, even the ones we do in Markdown...

# Oh, this is affected too?

We can use the properties of CSS that web developers use to avoid conflicts like these to target our changes to only the areas we intend. For example, the following will only affect `h1` headings with the `output` class on them:

In [209]:
display(HTML('''
<style>
    h1.output:hover {
      background-color: red;
    }
</style>
<h1 class='output'>Hello, world!</h1>
'''))

And if we want to get even more specific, we can add a containing div and use its classname or ID to fence in our changes:

In [210]:
display(HTML('''
<style>
    #block-1234 h1.output:hover {
      background-color: blue;
    }
</style>
<div id="block-1234">
    <h1 class='output'>Hello, world!</h1>
</div>
'''))

## We can insert Javascript too

The following is based on https://stackoverflow.com/questions/44349183/cant-run-d3js-to-a-website-jupyter-notebook but removes the direct use of `%%javascript` magic. Why? Well, whatever we can do directly in Python is something we can abstract into a function. This will be important for making it easy for others to insert these things into their notebooks without having to switch between multiple libraries. Note, there are more advanced integrations of D3 as well (https://github.com/ResidentMario/py_d3), which are great *if* you're comfortable switching back and forth between multiple languages. We want to *hide* that as much as possible. This way, we can use the power of D3 without the end user really knowing that they're using D3 or Javascript or HTML.

In [211]:
from IPython.display import Javascript
def notebook_init():
    display(Javascript('''
        require.config({
            paths: {
                d3: "https://d3js.org/d3.v4.min",
                d3_selection: "https://d3js.org/d3-selection-multi.v0.4.min"
             }
        });

        require(["d3"], function(d3) {
            window.d3 = d3;
        });
        require(["d3_selection"]);
    '''))
    
# If notebook_init() is in a library, people can simply call it, without knowing that they're actually doing a 
# fancy D3JS import
notebook_init()

<IPython.core.display.Javascript object>

In [212]:
HTML('''
<style>

.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis--x path {
  display: none;
}

</style>
<svg width="960" height="500"></svg>
<script>

var svg = d3.select("svg"),
    margin = {top: 20, right: 20, bottom: 30, left: 40},
    width = +svg.attr("width") - margin.left - margin.right,
    height = +svg.attr("height") - margin.top - margin.bottom;

var x = d3.scaleBand().rangeRound([0, width]).padding(0.1),
    y = d3.scaleLinear().rangeRound([height, 0]);

var g = svg.append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

d3.tsv("../data/raw/data.tsv", function(d) {
  d.frequency = +d.frequency;
  return d;
}, function(error, data) {
  if (error) throw error;

  x.domain(data.map(function(d) { return d.letter; }));
  y.domain([0, d3.max(data, function(d) { return d.frequency; })]);

  g.append("g")
      .attr("class", "axis axis--x")
      .attr("transform", "translate(0," + height + ")")
      .call(d3.axisBottom(x));

  g.append("g")
      .attr("class", "axis axis--y")
      .call(d3.axisLeft(y).ticks(10, "%"))
    .append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", "0.71em")
      .attr("text-anchor", "end")
      .text("Frequency");

  g.selectAll(".bar")
    .data(data)
    .enter().append("rect")
      .attr("class", "bar")
      .attr("x", function(d) { return x(d.letter); })
      .attr("y", function(d) { return y(d.frequency); })
      .attr("width", x.bandwidth())
      .attr("height", function(d) { return height - y(d.frequency); });
});

</script>
''')

In [226]:
def bouncing_balls(down_speed, up_speed):
    display(HTML(
    '<script>var down_speed = ' + str(down_speed) + '; var up_speed = ' + str(up_speed) + ';</script>' +
    '''
        <style>
          .ball {
            position: absolute;
            background-color: red;
            border-radius: 50px;
          }
        </style>
        <div class="bounce-container" style="width: 100%; height: 300px"></div>
        <script>
            var width = 300;
            var height = 200;
            var radius = 50; 
            var bounceTop = 0 + 'px';
            var bounceBottom = (height - radius * 2) + 'px';
            var ball = d3.select('.bounce-container')
                        .selectAll('div') 
                        .data([0])
                        .enter()
                        .append('div').attr('class', 'ball') 
                        .style("top", "bounceTop")
                        .style("left", width / 2 - radius + 'px')
                        .style("width", radius * 2 + 'px')
                        .style("height", radius * 2 + 'px')
            function intervalFunc(){ 
              ball.transition().duration(down_speed) 
              .style('top',height + 'px') 
              .transition().duration( up_speed ) 
              .style('top',0+'px')};
            if(document.bounceInterval) {
                clearInterval(document.bounceInterval);
            }
            document.bounceInterval = setInterval(intervalFunc, down_speed+up_speed);
        </script>

    '''))

In [228]:
bouncing_balls(500, 500)